In [1]:
# load libraries


import torch
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
import sys
import numpy as np
import torch

#plot a cv2 image
import cv2




In [2]:
# add modified YOLOv8 library (cloned from git: URL)


from ultralytics import YOLO #make sure you do not have another library with the same name

In [3]:
#Load pre-trained model
model = YOLO("forams4-1280-augmentedraw-bigtest-200epochs-yolov8x.pt")

#load example image
im0 = Image.open("sampleimg-0.jpg")
im1 = Image.open("sampleimg-1.jpg")
im2 = Image.open("sampleimg-2.jpg")
im3 = Image.open("sampleimg-3.jpg")

#run prediction with visualize = True to generate heatmap
#results = model.predict(source=im3, save=False, visualize = True)  # save plotted images

#The results are stored in /runs/detect/predict..

In [28]:
def labelreader(labpath=None):
        labels = []
        with open(labpath, 'r') as f:
            for line in f:
                #append each line to the labels as floats
                labels.append(line.strip())

        for i in range(len(labels)):
            labels[i] = labels[i].split(' ')
            for j in range(len(labels[i])):
                labels[i][j] = float(labels[i][j])

        #cast each element to float and store to tensor
        labels = torch.tensor(labels)
        return labels


def plot_heatmap(image, bboxes = None, savename = None, plot=False):
    
    cam_img = image
    
    if bboxes is not None:
        colors = [[0, 0, 255], [0, 255, 0], [255, 0, 0], [255,255,255]]
        boxlabels = ["sediment", "agglutinated", "calcareous", "planktic"]
        labheight = -10
        labwidth = -7
        imgsz = (image.shape[0],image.shape[1])
        for i in range(bboxes.shape[0]):
            box = bboxes[i,:]
            cls = box[0]
            cls = int(cls)
            xc = float(box[1])
            yc = float(box[2])
            w = float(box[3])
            h = float(box[4])
            x1 = round((xc-0.5*w)*imgsz[0])
            x2 = round((xc+0.5*w)*imgsz[0])
            y1 = round((yc-0.5*h)*imgsz[1])
            y2 = round((yc+0.5*h)*imgsz[1])
            
            cv2.rectangle(cam_img, (x1, y1), (x2, y2), colors[cls], 2)
            cv2.putText(cam_img, boxlabels[cls], (x1+labwidth, y1+labheight),cv2.FONT_HERSHEY_SIMPLEX, 0.8, colors[cls], 2)
    
    if savename is not None:
        cv2.imwrite(savename, cam_img)
    if plot:
        cv2.imshow("image",cam_img)
    
    return cam_img


In [ ]:
# Eigen CAM #

#from #https://github.com/rigvedrs/YOLO-V8-CAM/blob/main/YOLO%20v8n%20EigenCAM.ipynb

from yolo_cam.eigen_cam import EigenCAM
from yolo_cam.utils.image import show_cam_on_image, scale_cam_image

model = YOLO('forams4-1280-augmentedraw-bigtest-200epochs-yolov8x.pt')


In [36]:
sampleid = 3
samplename = "sampleimg-" + str(sampleid)

img = cv2.imread(samplename + ".jpg")
labpath = samplename + ".txt"
savename = samplename + "-eigencam.jpg"

#img = cv2.resize(img, (640, 640))
#img = cv2.resize(img, (1280, 1280))
rgb_img = img.copy()
img = np.float32(img) / 255
print(img.shape)
#plt.imshow(img)
#plt.show()

target_layers =[model.model.model[-4]]

cam = EigenCAM(model, target_layers,task='od')
grayscale_cam = cam(rgb_img)[0, :, :]
cam_image = show_cam_on_image(img, grayscale_cam, use_rgb=False)
#plt.imshow(cam_image)
#plt.show()

import matplotlib.pyplot as plt
g_scale = np.stack([grayscale_cam] * 3, axis=2)
#plt.imshow(g_scale)
#cv2.imshow(cam_image)

#im = cv2.cvtColor(rgb_img, cv2.COLOR_RGB2BGR)
im = rgb_img
Image.fromarray(np.hstack((im, cam_image)))

print(cam_image.shape)

#load labels into useable format
bboxes = labelreader(labpath)

eigenimg = plot_heatmap(cam_image, bboxes = bboxes, savename = savename)

(1280, 1280, 3)


0: 1280x1280 1 calcareous, 33 planktics, 3058.3ms
Speed: 4.1ms preprocess, 3058.3ms inference, 0.6ms postprocess per image at shape (1, 3, 1280, 1280)


(1280, 1280, 3)


: 

In [3]:
# This block defines some useful functions

# define a function that reads the labels from the label file
def labelreader(labpath=None):
        labels = []
        with open(labpath, 'r') as f:
            for line in f:
                #append each line to the labels as floats
                labels.append(line.strip())

        for i in range(len(labels)):
            labels[i] = labels[i].split(' ')
            for j in range(len(labels[i])):
                labels[i][j] = float(labels[i][j])

        #cast each element to float and store to tensor
        labels = torch.tensor(labels)
        return labels


#This function generates an image where the CAM heatmap has been overlayed onto the original image
def make_heatmap(image, heatmap, transparancy = 0.3, plot=False):
    heatmapp = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
    
    imgg = image.astype(np.uint8)
    cam_img = transparancy * heatmapp + (1-transparancy) * imgg
    cam_img = cam_img.astype(np.uint8)
    
    if plot:
        cv2.imshow("image",cam_img)
    
    return cam_img



# This function plots the heatmap and the bounding boxes on the same image
def plot_heatmap(image, heatmap, bboxes = None, transparancy = 0.3, savename = None):
    
    cam_img = make_heatmap(image, heatmap, transparancy, plot=False)
    
    if bboxes is not None:
        colors = [[0, 0, 255], [0, 255, 0], [255, 0, 0], [255,255,255]]
        boxlabels = ["sediment", "agglutinated", "calcareous", "planktic"]
        labheight = -10
        labwidth = -7
        imgsz = (image.shape[0],image.shape[1])
        for i in range(bboxes.shape[0]):
            box = bboxes[i,:]
            cls = box[0]
            cls = int(cls)
            xc = float(box[1])
            yc = float(box[2])
            w = float(box[3])
            h = float(box[4])
            x1 = round((xc-0.5*w)*imgsz[0])
            x2 = round((xc+0.5*w)*imgsz[0])
            y1 = round((yc-0.5*h)*imgsz[1])
            y2 = round((yc+0.5*h)*imgsz[1])
            
            cv2.rectangle(cam_img, (x1, y1), (x2, y2), colors[cls], 2)
            cv2.putText(cam_img, boxlabels[cls], (x1+labwidth, y1+labheight),cv2.FONT_HERSHEY_SIMPLEX, 0.8, colors[cls], 2)
    
    if savename is not None:
        cv2.imwrite(savename, cam_img)
    cv2.imshow("image",cam_img)
    
    return cam_img



In [10]:
# lets reload the image as a cv2 image
image = cv2.imread("sampleimg-3.jpg")

#give the path to the label file of the sample image
labpath = "sampleimg-3.txt"

#load labels into useable format
bboxes = labelreader(labpath)

In [11]:
# Lets import the heatmap from the 

run = 4 #run number for the prediction
# 0: ""
# 1: 2
# 2: 3
# 3: 4

layer = 21 #which layer to use for the heatmap

resultfolder = "./runs/detect/"+ "predict" + str(run) + "/"

#load heatmap
heatmap = np.load(resultfolder + str(layer) + ".npy")


#run the function
savename = "sampleimg"+str(run)+"-heatmap.jpg"
cam_img = plot_heatmap(image,heatmap, bboxes, transparancy = 0.3, savename = savename)


In [1]:
# Eigen CAM #

#from #https://github.com/rigvedrs/YOLO-V8-CAM/blob/main/YOLO%20v8n%20EigenCAM.ipynb

from yolo_cam.eigen_cam import EigenCAM
from yolo_cam.utils.image import show_cam_on_image, scale_cam_image


#load model
model = YOLO('forams4-1280-augmentedraw-bigtest-200epochs-yolov8x.pt')

img = cv2.imread('sampleimg-0.jpg')
#rgb_img = img.copy()
rgb_img = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
cv2.imshow("img",img)
#plt.show()

img = np.float32(img) / 255
print(img.shape)


#target_layers =[model.model.model[-4]]
target_layers = [model.model.model[21]]

NameError: name 'YOLO' is not defined

In [21]:

img = np.float32(img) / 255
print(img.shape)


#target_layers =[model.model.model[-4]]
target_layers = [model.model.model[21]]

cam = EigenCAM(model, target_layers,task='od')
grayscale_cam = cam(rgb_img)[0, :, :]
cam_image = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)


import matplotlib.pyplot as plt
g_scale = np.stack([grayscale_cam] * 3, axis=2)
#plt.imshow(g_scale)

im = cv2.cvtColor(rgb_img, cv2.COLOR_RGB2BGR)
Image.fromarray(np.hstack((im, cam_image)))


(1280, 1280, 3)


 YOLO call 




 predict




 type(self.predictor)
 
<class 'ultralytics.yolo.v8.detect.predict.DetectionPredictor'>
<class 'ultralytics.nn.autobackend.AutoBackend'>





 basepredictor __call__ 

<class 'ultralytics.nn.autobackend.AutoBackend'>





 basepredictor stream_inference 




 batch#
1





 batch reqgrad

False


 autobackend forward 


b, ch, h, w 

1 3 1280 1280
autobackend self.pt or self.nn_module 

<class 'ultralytics.nn.tasks.DetectionModel'>
predict once
<class 'ultralytics.nn.modules.conv.Conv'>
<class 'ultralytics.nn.modules.conv.Conv'>
<class 'ultralytics.nn.modules.block.C2f'>
<class 'ultralytics.nn.modules.conv.Conv'>
<class 'ultralytics.nn.modules.block.C2f'>
<class 'ultralytics.nn.modules.conv.Conv'>
<class 'ultralytics.nn.modules.block.C2f'>
<class 'ultralytics.nn.modules.conv.Conv'>
<class 'ultralytics.nn.modules.block.C2f'>
<class 'ultralytics.nn.modules.block.SPPF'>
<class 'torch.nn.modules.upsampling.Upsample'>
<class 'ultralytics.nn.mod

0: 1280x1280 34 Sediments, 9 planktics, 4451.6ms
Speed: 8.5ms preprocess, 4451.6ms inference, 224.2ms postprocess per image at shape (1, 3, 1280, 1280)




 visualize, save, write 




Exception: The input image should np.float32 in the range [0, 1]